
# Job Info Extraction — Notebook Test

This notebook exercises `src/modules/job_info_extraction` end‑to‑end:

- Source discovery & resume safety
- Domain selection (taxonomy tournament + cache)
- Required skills extraction (domain‑scoped, allowed‑skills CSV)
- JSONL/CSV outputs + submissions bundle
- Quick inspection & troubleshooting helpers


In [1]:
import sys
sys.path.append("..")

In [2]:
from pathlib import Path
from pprint import pprint
import json
import os

# Pipeline
from src.modules.job_info_extraction.pipeline import ExtractJobsConfig, extract_jobs

# (Optional) helpers for preflight / debugging
from src.modules.job_info_extraction.io import discover_sources, load_resume_set

# Optional KG smoke test (jobs-only)
try:
    from src.modules.knowledge_graph.utils import load_bundle
    from src.modules.knowledge_graph.graph import build_kg
except Exception:
    load_bundle = None
    build_kg = None

# --- Paths (adjust if needed) ---
DATA_JOBS_DIR        = Path("../../GDSC-8/data/jobs")
OUT_DIR              = Path("../processed_data/outputs/jobs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Taxonomy for domain tournament (optional but recommended)
TAXONOMY_CSV         = Path("../processed_data/unix_cr_gd_dp_v460_utf8_translated.csv")

# Allowed skills CSV (domain -> allowed skill list; recommended)
ALLOWED_SKILLS_CSV   = Path("../processed_data/outputs/domain_allowed_skills_2025-10-21_adjusted.csv")

# Optional domain settings JSON (used in enrichment, safe to be missing)
DOMAIN_SETTINGS_JSON = Path("../processed_data/outputs/domain_settings.json")

# The pipeline will write:
JSONL_PATH = OUT_DIR / "jobs.jsonl"
CSV_PATH   = OUT_DIR / "jobs.csv"

print("Jobs dir exists:", DATA_JOBS_DIR.exists())
print("Taxonomy CSV   :", TAXONOMY_CSV.exists(), TAXONOMY_CSV)
print("Allowed skills :", ALLOWED_SKILLS_CSV.exists(), ALLOWED_SKILLS_CSV)
print("Out dir        :", OUT_DIR.resolve())

Jobs dir exists: True
Taxonomy CSV   : True ../processed_data/unix_cr_gd_dp_v460_utf8_translated.csv
Allowed skills : False ../processed_data/outputs/domain_allowed_skills_2025-10-21_adjusted.csv
Out dir        : /home/ec2-user/SageMaker/GDSC/processed_data/outputs/jobs


In [3]:
from src.modules.training_info_extraction.llm_client import call_llm


print("MISTRAL_API_KEY set:", bool(os.getenv("MISTRAL_API_KEY")))
MODEL = "mistral-medium-latest"

# Tiny ping: ask the LLM for a trivial JSON; we just want a non-None response
ping = call_llm('Return ONLY this: {"ok": true}', model=MODEL)
print("LLM ping returned:", type(ping), ping.keys() if isinstance(ping, dict) else ping)

MISTRAL_API_KEY set: True
LLM ping returned: <class 'dict'> dict_keys(['content', 'model', 'duration', 'input_tokens', 'output_tokens', 'total_tokens'])


In [4]:
sources = discover_sources(input_dir=DATA_JOBS_DIR, inputs=None, max_files=None)
resume_ids = load_resume_set(JSONL_PATH) if JSONL_PATH.exists() else set()

to_process = [p for p in sources if p.stem not in resume_ids]

print("📁 jobs dir      :", DATA_JOBS_DIR.resolve())
print("🧾 JSONL         :", JSONL_PATH.resolve())
print("🔍 discovered    :", len(sources))
print("↩️  resume entries:", len(resume_ids))
print("▶️  to process    :", len(to_process))
print("preview          :", [p.name for p in to_process[:8]])

📁 jobs dir      : /home/ec2-user/SageMaker/GDSC-8/data/jobs
🧾 JSONL         : /home/ec2-user/SageMaker/GDSC/processed_data/outputs/jobs/jobs.jsonl
🔍 discovered    : 200
↩️  resume entries: 130
▶️  to process    : 200
preview          : ['j0.md', 'j1.md', 'j10.md', 'j100.md', 'j101.md', 'j102.md', 'j103.md', 'j104.md']


In [30]:
cfg = ExtractJobsConfig(
    input_dir=DATA_JOBS_DIR,
    out_dir=OUT_DIR,
    model="mistral-small-latest",  # your Mistral model
    resume=True,                   # skip already-processed sources
    max_files=None,
    sleep_seconds=0.0,

    # Domain tournament (optional if TAXONOMY_CSV missing)
    taxonomy_csv=TAXONOMY_CSV if TAXONOMY_CSV.exists() else None,

    # Strict required-skills (domain-scoped) based on an allowed-skills table:
    extract_required_skills=True,
    allowed_skills_csv=ALLOWED_SKILLS_CSV if ALLOWED_SKILLS_CSV.exists() else None,

    # Not required for per-job extraction; used by enrichment flows:
    domain_settings_json=DOMAIN_SETTINGS_JSON if DOMAIN_SETTINGS_JSON.exists() else None,
)

result = extract_jobs(cfg)
pprint(result)

ValueError: education_level is required and must be one of EDU_LEVELS.

In [25]:
import pandas as pd

assert CSV_PATH.exists(), "CSV not found—pipeline should always create it (headers at least)."
df = pd.read_csv(CSV_PATH, dtype=str, encoding="utf-8").fillna("")
display(df.head(20))

bundle_path = Path(result["bundle"])
job_map = json.loads(bundle_path.read_text(encoding="utf-8"))
print("Bundle rows:", len(job_map))

# Inspect a sample job structure
sample = next(iter(job_map.values())) if job_map else None
if sample:
    print("\nSAMPLE JOB")
    print("title       :", sample.get("title"))
    print("domain      :", sample.get("domain"))
    print("work_type   :", sample.get("work_type"))
    print("languages   :", sample.get("languages_required"))
    print("req skills  :", sample.get("required_skills"))
    # NOTE: 'skills_evidence' is only in JSONL, not necessarily in the bundle.
else:
    print("No jobs present in bundle.")

,job_id,source_key,title,domain,location_city,work_type,languages_required,education_level_required,years_of_experience_required,required_skills,job_role,domain_confidence,rome_title_en,rome_confidence,source_path
0,job_j0,,Accounting Intern – Bookkeeping & Admin,Accounting & Management,Brasília,onsite,"pt-br,en",Graduação,2.0,tax regulations and compliance processes(Inter...,,,,,
1,job_j1,,Accounting Trainee – Risk & Data Focus,Accounting & Management,,remote,pt-br,Tecnólogo,1.0,risk evaluation(Basic); data analysis(Basic),,,,,
2,job_j10,,Banking Operations Analyst,unspecified,,remote,pt-br,Graduação,2.0,coordinating between departments(Intermediate)...,,,,,
3,job_j100,,Legal Intern – Negotiation & Research,Law,Salvador,onsite,pt-br,Técnico,3.0,Negotiation(Advanced); Legal Research(None); L...,,,,,
4,job_j101,,Legal Assistant – Statutory Interpretation,Law,Fortaleza,onsite,pt-br,Graduação,3.0,Statutory Interpretation(Advanced); Legal Rese...,,,,,
5,job_j102,,Junior Legal Specialist – Statutory Review,Law,,remote,pt-br,Técnico,1.0,statutory analysis(Intermediate); legal interp...,,,,,
6,job_j103,,Legal Writing Assistant – Case Support,Law,Recife,onsite,"pt-br,en",Graduação,2.0,Legal Writing(None); Legal Document Preparatio...,,,,,
7,job_j104,,Contracts & Research Intern,Law,Curitiba,unspecified,pt-br,Tecnólogo,3.0,Contract drafting(Advanced); Legal research(In...,,,,,
8,job_j105,,Litigation Procedures Analyst,Law,Recife,onsite,"pt-br,en",Graduação,2.0,case management systems(None); legal research ...,,,,,
9,job_j106,,Junior Legal Analyst – Ethics & Drafting,Law,Curitiba,onsite,"pt-br,en",Técnico,,legal writing and document drafting(None); leg...,,,,,


NameError: name 'result' is not defined

### Prefill & Backfill

In [ ]:
from pathlib import Path
from src.modules.job_info_extraction.prefill import prefill_bundle_from_csvs, PrefillInputs

labels_csv = Path("../processed_data/jobs_extracted_labels_en_blocked.csv")
skills_csv = Path("../processed_data/outputs/jobs_skills_2025-10-21_2.csv")  # adjust to your file name

out_bundle = Path("../submissions/extracted_jobs_prefill.json")
bundle = prefill_bundle_from_csvs(PrefillInputs(labels_csv, skills_csv), out_path=out_bundle)
len(bundle), out_bundle

In [ ]:
from pathlib import Path
from src.modules.job_info_extraction.backfill import BackfillConfig, backfill_missing_fields

cfg = BackfillConfig(
    job_bundle_path=Path("../submissions/extracted_jobs_prefill.json"),
    job_md_dir=Path("../data/jobs"),
    out_path=Path("../submissions/extracted_jobs_prefill_backfilled.json"),
    model="mistral-small-latest",
    fields_considered=None,  # default set (company, location_city, work_type, languages_required, education_level_required, years_of_experience_required, required_skills)
    resume_json=Path("../processed_data/outputs/jobs/backfill_resume.json"),  # optional, resume-friendly
)
backfill_missing_fields(cfg)

In [ ]:
from pathlib import Path
from src.modules.job_info_extraction.backfill import merge_bundles_only_if_none

merge_bundles_only_if_none(
    primary_path=Path("../submissions/extracted_jobs_2025-10-28.json"),
    secondary_path=Path("../submissions/extracted_jobs_prefill_backfilled.json"),
    out_path=Path("../submissions/extracted_jobs_merged_2025-10-28.json"),
    include_secondary_only=False,  # your original rule
)